<a href="https://colab.research.google.com/github/Jhonald73/TPproj_github/blob/main/W2S2_Activities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import userdata
from openai import OpenAI

# Load key from Google Colab Secrets
api_key = userdata.get('OPENAI_API_KEY')

client = OpenAI(
    # This is the default and can be omitted
    api_key=api_key,
)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Automated Data Prep/laptop_prices_2024_sgd_TL.csv")

In [7]:
# Convert the first few rows to a string to send to OpenAI
data_preview = df.head(10).to_csv(index=False)
print(data_preview)

response = client.responses.create(
    model="gpt-4o-mini",
    instructions="You are an expert data scientist with extensive knowledge of predictive analysis and linear regression.",
    input=f"Dataset: laptop_prices_2024_sgd_TL (150 samples, 4 features, 3 classes)\nHere are the first 10 rows of the dataset:\n{data_preview}]",
)
print(response.output_text)

Brand,Model,CPU,GPU,RAM_GB,Storage_Type,Storage_GB,Touchscreen,Weight_kg,Screen_Size_inch,Discount_percent,Price_SGD,Brand_Discount,Member_Discount
Acer,Aspire 5,Intel i9-14900HK,NVIDIA RTX 4070,64,SSD,256,False,1.56,16.0,3.27,2413.36,5,144.8
Acer,Nitro 5,AMD Ryzen 9 8900HX,AMD Radeon 780M,32,SSD,1024,True,1.45,14.0,5.03,1773.75,5,124.16
Acer,Nitro 5,AMD Ryzen 5 8600H,NVIDIA RTX 4050,32,SSD,2048,False,1.34,14.0,4.41,1634.07,5,98.04
Acer,TravelMate P6,Intel Core Ultra 7 15500H,NVIDIA RTX 4060,16,SSD,4096,True,1.18,13.3,2.16,2362.59,5,118.13
Acer,Predator Helios 300,Intel i7-14800H,NVIDIA RTX 4070,8,SSD,1024,True,1.31,14.0,6.93,2218.55,5,155.3
Acer,Aspire 5,Intel i9-14900HK,NVIDIA RTX 4080,32,SSD,256,False,3.34,17.3,8.94,2224.12,5,155.69
Acer,TravelMate P6,Intel i9-14900HK,NVIDIA RTX 4080,128,SSD,1024,True,1.11,13.3,11.72,3016.36,5,211.15
Acer,Predator Helios 300,Intel Core Ultra 9 15700H,NVIDIA RTX 4070,16,SSD,2048,True,1.29,13.3,8.84,2579.29,5,180.55
Acer,Swift 3,Intel i5-14500H,NVIDIA

Brand,Model,CPU,GPU,RAM_GB,Storage_Type,Storage_GB,Touchscreen,Weight_kg,Screen_Size_inch,Discount_percent,Price_SGD,Brand_Discount,Member_Discount
Acer,Aspire 5,Intel i9-14900HK,NVIDIA RTX 4070,64,SSD,256,False,1.56,16.0,3.27,2413.36,5,144.8
Acer,Nitro 5,AMD Ryzen 9 8900HX,AMD Radeon 780M,32,SSD,1024,True,1.45,14.0,5.03,1773.75,5,124.16
Acer,Nitro 5,AMD Ryzen 5 8600H,NVIDIA RTX 4050,32,SSD,2048,False,1.34,14.0,4.41,1634.07,5,98.04
Acer,TravelMate P6,Intel Core Ultra 7 15500H,NVIDIA RTX 4060,16,SSD,4096,True,1.18,13.3,2.16,2362.59,5,118.13
Acer,Predator Helios 300,Intel i7-14800H,NVIDIA RTX 4070,8,SSD,1024,True,1.31,14.0,6.93,2218.55,5,155.3
Acer,Aspire 5,Intel i9-14900HK,NVIDIA RTX 4080,32,SSD,256,False,3.34,17.3,8.94,2224.12,5,155.69
Acer,TravelMate P6,Intel i9-14900HK,NVIDIA RTX 4080,128,SSD,1024,True,1.11,13.3,11.72,3016.36,5,211.15
Acer,Predator Helios 300,Intel Core Ultra 9 15700H,NVIDIA RTX 4070,16,SSD,2048,True,1.29,13.3,8.84,2579.29,5,180.55
Acer,Swift 3,Intel i5-14500H,NVIDIA RTX 4050,64,SSD,4096,False,1.4,13.3,6.92,1976.94,5,98.85
Acer,Nitro 5,Intel Core Ultra 9 15700H,NVIDIA RTX 4070,16,SSD,2048,False,1.15,13.3,5.25,2538.34,5,152.3

Based on the dataset provided, let's explore key aspects for predictive analysis and linear regression. You have several features related to laptop specifications and pricing that can be useful for modeling.

### Features Overview
1. **Brand**
2. **Model**
3. **CPU**
4. **GPU**
5. **RAM_GB**
6. **Storage_Type**
7. **Storage_GB**
8. **Touchscreen** (boolean)
9. **Weight_kg**
10. **Screen_Size_inch**
11. **Discount_percent**
12. **Price_SGD**
13. **Brand_Discount**
14. **Member_Discount**

### Steps for Analysis

1. **Data Preprocessing:**
   - **Encoding Categorical Variables:** Convert categorical features like `Brand`, `Model`, `CPU`, `GPU`, and `Storage_Type` into numerical form using techniques like one-hot encoding or label encoding.
   - **Handling Missing Values:** Check for any missing values and decide how to handle them (e.g., imputation, removal).
   - **Normalizing/Standardizing Numerical Features:** Apply scaling to the features, especially those that vary in magnitude, such as `Weight_kg`, `Screen_Size_inch`, and `Price_SGD`.

2. **Exploratory Data Analysis (EDA):**
   - **Correlation Analysis:** Use correlation matrices to understand relationships between features and the target variable (`Price_SGD`).
   - **Visualizations:** Create scatter plots, box plots, and histograms to visualize the distribution of laptop prices and other features.

3. **Feature Selection:**
   - Assess which features are most influential for predicting laptop prices. This may involve using techniques such as recursive feature elimination or considering models that indicate feature importance.

4. **Building the Model:**
   - **Linear Regression:** Fit a linear regression model to the data. Depending on feature relationships, you might want to explore polynomial regression or transformations (e.g., log transformation for price).
   - **Train-Test Split:** Divide the dataset into training and testing sets (e.g., 70-30 split) to evaluate model performance.

5. **Model Evaluation:**
   - Assess the model using metrics like R-squared, Mean Absolute Error (MAE), and Root Mean Squared Error (RMSE) on the test set.
   - Use residual plots to check for homoscedasticity and linearity.

6. **Tuning & Optimization:**
   - Experiment with regularization techniques (e.g., Lasso, Ridge regression) to improve model performance.
   - Consider cross-validation for more robust evaluation of model stability.

### Implementation Example (Python Pseudocode)

Here’s a basic pseudocode outline you might follow in Python:

```python
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Load dataset
data = pd.read_csv('laptop_prices_2024_sgd_TL.csv')

# Data Preprocessing
data = data.dropna()  # Handle missing values
data = pd.get_dummies(data, columns=['Brand', 'Model', 'CPU', 'GPU', 'Storage_Type'], drop_first=True)

# Feature and target selection
X = data.drop('Price_SGD', axis=1)  # Features
y = data['Price_SGD']  # Target

# Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Model training
model = LinearRegression()
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')
```

### Conclusion
Feel free to adjust the analysis depending on the insights derived from EDA or the performance of the initial models. Linear regression offers a foundation for predictive analysis, but you might consider more complex models depending on the accuracy required.

What parts of the code from APREP are specific to the dataset?
Answer: "input" code.

What parts needs to be modified to adapt to the new dataset?
Answer:You need a code to read a dataset from specific directory where its saved.

Is there a way to make it generic and adaptable to any new dataset?
Answer. A function call below

import pandas as pd

def load_dataset(path, target_col=None):
    df = pd.read_csv(path)


